In [1]:
# Load dependencies
from bs4 import BeautifulSoup
import requests
import json
import time
import random
import webbrowser
import sys

sys.setrecursionlimit(9999)

In [ ]:
# Get the function parameters from terminal
# url = sys.argv[1]
# category = int(sys.argv[2])

In [18]:
# Load url
# url = "https://www.ticketswap.nl/event/queen-adam-lambert-the-rhapsody-tour/floor-tickets/23a41750-3429-4ae9-9727-2ec36f43bf4f/1449038"
# url = "https://www.ticketswap.nl/event/korn/floor-tickets/f1ec04f3-af13-4e99-af2a-6c53bb4614a2/1504712"
url = "https://www.ticketswap.nl/event/korn/f1ec04f3-af13-4e99-af2a-6c53bb4614a2"

In [11]:
def check_ticketswap_offers(url, category, counter = 0):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    ldjson_soup = soup.findAll("script", attrs = {'type':"application/json"})
    ldjson_str = ldjson_soup[0].text
    ldjson = json.loads(ldjson_str)
    #if int(ldjson['props']['pageProps']['data']['node']['availableTicketsCount']) < 1:
    if int(ldjson['props']['pageProps']['eventType']['availableTicketsCount']) < 1:
        seconds = random.randrange (75, 125) / 100
        time.sleep(seconds)
        counter += 1
        print("Try %s" % counter)
        return check_ticketswap_offers(url, category, counter)
    url_open = ldjson['props']['pageProps']['data']['node']['event']['types']['edges'][category-1]['node']['availableListings']['edges'][0]['node']['uri']['url']
    webbrowser.open_new(url_open)
    
#Tip: use ldjson.keys() to see set of keys on the highest level

In [12]:
#TODO: come up with a way to scan ticket types and select via prompt to create dynamic code

In [13]:
# Load & test url:
check_ticketswap_offers(url, 1)

KeyError: 'data'

Sub-ticket page research:

In [20]:
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
ldjson_soup = soup.findAll("script", attrs = {'type':"application/json"})
ldjson_str = ldjson_soup[0].text
ldjson = json.loads(ldjson_str)

In [15]:
ldjson.keys()

dict_keys(['props', 'page', 'query', 'buildId', 'assetPrefix', 'runtimeConfig', 'isFallback'])

In [21]:
ldjson['props']['pageProps']['eventType'].keys()

KeyError: 'eventType'

In [17]:
ldjson['props']['pageProps']['eventType']['availableListings']['edges']

[{'node': {'seller': {'id': 'VXNlcjo3ODk1OTI=',
    'avatar': 'https://cdn.ticketswap.com/public/201806/6d5fca94-c2b4-4bef-8a41-3fccb9c321e1.jpeg',
    '__typename': 'PublicUser'},
   '__typename': 'PublicListing'},
  '__typename': 'ListingEdge'},
 {'node': {'seller': {'id': 'VXNlcjo4MzgwNQ==',
    'avatar': 'https://cdn.ticketswap.com/public/201806/4e1cae78-57d5-4491-b9e0-db96dd47fbd8.jpeg',
    '__typename': 'PublicUser'},
   '__typename': 'PublicListing'},
  '__typename': 'ListingEdge'}]

Top-ticket page research:

In [98]:
url = 'https://www.ticketswap.com/event/the-weeknd-the-after-hours-tour/ea3cfc09-a4c7-44d1-afb3-07aac5e14551'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
ldjson_soup = soup.findAll("script", attrs = {'type':"application/json"})
ldjson_str = ldjson_soup[0].text
ldjson = json.loads(ldjson_str)

In [28]:
ldjson.keys()

dict_keys(['props', 'page', 'query', 'buildId', 'assetPrefix', 'runtimeConfig', 'isFallback'])

In [29]:
ldjson['props'].keys()

dict_keys(['initialI18nStore', 'initialLanguage', 'i18nServerInstance', 'viewer', 'deviceInfo', 'pageProps', 'languageFromRequest', 'apolloState'])

In [94]:
ldjson['props']['apolloState']

{'RXZlbnRUeXBlOjE2MjM5NzY=': {'id': 'RXZlbnRUeXBlOjE2MjM5NzY=',
  'slug': 'regular-tickets',
  'title': 'Regular',
  'isSellingBlocked': False,
  'startDate': '2020-03-14T19:30:00+01:00',
  'endDate': None,
  'availableTicketsCount': 1,
  'soldTicketsCount': 0,
  'ticketAlertsCount': 0,
  'organizerProduct': None,
  'listings({"filter":{"listingStatus":"AVAILABLE"},"first":2})': {'type': 'id',
   'generated': True,
   'id': '$RXZlbnRUeXBlOjE2MjM5NzY=.listings({"filter":{"listingStatus":"AVAILABLE"},"first":2})',
   'typename': 'ListingConnection'},
  'listings({"filter":{"listingStatus":"SOLD"},"first":2})': {'type': 'id',
   'generated': True,
   'id': '$RXZlbnRUeXBlOjE2MjM5NzY=.listings({"filter":{"listingStatus":"SOLD"},"first":2})',
   'typename': 'ListingConnection'},
  'ticketAlerts({"first":8})': {'type': 'id',
   'generated': True,
   'id': '$RXZlbnRUeXBlOjE2MjM5NzY=.ticketAlerts({"first":8})',
   'typename': 'TicketAlertConnection'},
  '__typename': 'EventType',
  'event': {'t

In [99]:

urls = [ldjson['props']['apolloState'][x]['url'] for x in ldjson['props']['apolloState'].keys() if x.endswith('.uri')]

In [100]:
ticketUrls = [url for url in urls if 'utm_source=ticketswap' in url]
print(ticketUrls)

['https://www.ticketswap.com/listing/the-weeknd-the-after-hours-tour/4626755/e7a67509a6?utm_campaign=publishedListingShare&utm_source=ticketswap&utm_medium=manualshare', 'https://www.ticketswap.com/listing/the-weeknd-the-after-hours-tour/4648025/5eb4d9ddf0?utm_campaign=publishedListingShare&utm_source=ticketswap&utm_medium=manualshare']
